In [326]:
import sys, os
sys.path.append(os.path.realpath(".."))

import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp

import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import pywt
import tsfresh.feature_extraction.feature_calculators as feats
import constants
import clinical_text_analysis as cta
reload(cta)
import tsfresh

from os import path

reload(read)
import wf_analysis.datasets as wfdata
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler

In [415]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix

In [4]:
trainData, trainGenders = pkl.load(open("../trainGenderData.pkl", 'rb'))
testData, testGenders = pkl.load(open("../testGenderData.pkl", 'rb'))

In [308]:
ss = StandardScaler()
trainData = ss.fit_transform(trainData)
testData = ss.fit_transform(testData)

In [230]:
trainData.shape[1]

69

In [406]:
model = Sequential([Dense(trainData.shape[1]*10, activation='tanh'),Dense(trainData.shape[1]*10, activation='tanh'),Dropout(0.5), Dense(2, activation='sigmoid')])
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["binary_accuracy"])
earlyStopping = EarlyStopping(monitor="val_binary_accuracy", patience=50, mode='max', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_binary_accuracy', save_best_only=True, mode='max', verbose=1)


In [407]:
history=model.fit(trainData, to_categorical(trainGenders), epochs=500, validation_split=0.1, callbacks=[earlyStopping, mc])

Train on 2356 samples, validate on 262 samples
Epoch 1/500
2356/2356 [==============================] - 4s 2ms/step - loss: 0.7132 - binary_accuracy: 0.5427 - val_loss: 0.6787 - val_binary_accuracy: 0.5420

Epoch 00001: val_binary_accuracy improved from -inf to 0.54198, saving model to best_model.h5
Epoch 2/500
2356/2356 [==============================] - 1s 212us/step - loss: 0.6779 - binary_accuracy: 0.5380 - val_loss: 0.6829 - val_binary_accuracy: 0.5344

Epoch 00002: val_binary_accuracy did not improve from 0.54198
Epoch 3/500
2356/2356 [==============================] - 0s 163us/step - loss: 0.6743 - binary_accuracy: 0.5484 - val_loss: 0.6980 - val_binary_accuracy: 0.5000

Epoch 00003: val_binary_accuracy did not improve from 0.54198
Epoch 4/500
2356/2356 [==============================] - 0s 176us/step - loss: 0.6697 - binary_accuracy: 0.5308 - val_loss: 0.6170 - val_binary_accuracy: 0.5592

Epoch 00004: val_binary_accuracy improved from 0.54198 to 0.55916, saving model to best_m


Epoch 00036: val_binary_accuracy did not improve from 0.56298
Epoch 37/500
2356/2356 [==============================] - 0s 157us/step - loss: 0.5537 - binary_accuracy: 0.6101 - val_loss: 0.7923 - val_binary_accuracy: 0.5477

Epoch 00037: val_binary_accuracy did not improve from 0.56298
Epoch 38/500
2356/2356 [==============================] - 0s 157us/step - loss: 0.5442 - binary_accuracy: 0.6104 - val_loss: 0.7456 - val_binary_accuracy: 0.5363

Epoch 00038: val_binary_accuracy did not improve from 0.56298
Epoch 39/500
2356/2356 [==============================] - 1s 216us/step - loss: 0.5441 - binary_accuracy: 0.6146 - val_loss: 0.6846 - val_binary_accuracy: 0.5649

Epoch 00039: val_binary_accuracy improved from 0.56298 to 0.56489, saving model to best_model.h5
Epoch 40/500
2356/2356 [==============================] - 0s 178us/step - loss: 0.5542 - binary_accuracy: 0.6051 - val_loss: 0.7458 - val_binary_accuracy: 0.5401

Epoch 00040: val_binary_accuracy did not improve from 0.56489
Ep

2356/2356 [==============================] - 0s 168us/step - loss: 0.4129 - binary_accuracy: 0.6842 - val_loss: 0.8961 - val_binary_accuracy: 0.4943

Epoch 00073: val_binary_accuracy did not improve from 0.59542
Epoch 74/500
2356/2356 [==============================] - 0s 164us/step - loss: 0.4090 - binary_accuracy: 0.6944 - val_loss: 0.9877 - val_binary_accuracy: 0.4943

Epoch 00074: val_binary_accuracy did not improve from 0.59542
Epoch 75/500
2356/2356 [==============================] - 0s 165us/step - loss: 0.3987 - binary_accuracy: 0.6906 - val_loss: 1.0199 - val_binary_accuracy: 0.5000

Epoch 00075: val_binary_accuracy did not improve from 0.59542
Epoch 76/500
2356/2356 [==============================] - 0s 172us/step - loss: 0.3992 - binary_accuracy: 0.6916 - val_loss: 0.9860 - val_binary_accuracy: 0.4714

Epoch 00076: val_binary_accuracy did not improve from 0.59542
Epoch 77/500
2356/2356 [==============================] - 0s 178us/step - loss: 0.4128 - binary_accuracy: 0.7018 

In [408]:
model = load_model('best_model.h5')

# Evaluating Model
On both train to see if we learned and test to see performance

In [409]:
y_pred = model.predict(trainData)

In [410]:
accuracy_score(trainGenders, y_pred.argmax(axis=1))

0.6684491978609626

In [411]:
roc_auc_score(trainGenders, y_pred.argmax(axis=1))

0.6679891400771307

Looking at test performance

In [412]:
y_pred = model.predict(testData)

In [413]:
accuracy_score(testGenders, y_pred.argmax(axis=1))

0.7290575916230366

In [414]:
roc_auc_score(testGenders, y_pred.argmax(axis=1))

0.6739243220807969

In [418]:
pd.DataFrame(confusion_matrix(testGenders, y_pred.argmax(axis=1)), index=["Female", "Male"], columns=["Predict Female", "Predict Male"])

,Predict Female,Predict Male
Female,115,93
Male,114,442
